In [1]:
import cv2
import os
from matplotlib import pyplot as plt
from predict import *
from getX import getX
from getY import getY
from resize import resize
from PIL import Image, ImageDraw

# data_path = "/media/queahren/DATA/服务外包大赛数据更新/训练数据/img_train_折线"
# data_path = "/media/queahren/DATA/DATA_A28_same/images"
data_path = "/media/queahren/DATA/testData/测试数据/images"
# data_path = "/media/queahren/DATA/MYDATA/images"

# img = cv2.imread("./220/draw.png")
# gray = cv2.imread("./220/draw.png", cv2.IMREAD_GRAYSCALE)

def predImage(img_path):
    assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
    img = Image.open(img_path)
    return predict(img)


# 不算最后一个数据
def calP(y_nums):
    
    y_nums.sort(key=lambda tup: tup[0], reverse=True)
    if len(y_nums) > 2:
        dy = y_nums[0][0] - y_nums[len(y_nums)-2][0]
        dy_p = y_nums[0][1] - y_nums[len(y_nums)-2][1]
        p = dy/dy_p
    elif len(y_nums) == 2:
        dy = y_nums[0][0] - y_nums[1][0]
        dy_p = y_nums[0][1] - y_nums[1][1]
        p = dy/dy_p
    else:
        print("error")
    return p, y_nums[0]

filename = os.listdir(data_path)
base_dir = data_path + "/"
font=cv2.FONT_HERSHEY_SIMPLEX

for imgname in filename:
    if(imgname.endswith('.png')):

        print(imgname)

        img = cv2.imread(base_dir + imgname)
        gray = cv2.imread(base_dir + imgname, cv2.IMREAD_GRAYSCALE)

        mean, std = cv2.meanStdDev(gray)
        if mean < 50:
            gray = 255 - gray

        bw = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 25)
        img2, ctrs, hier = cv2.findContours(
            bw.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        rects = [cv2.boundingRect(ctr) for ctr in ctrs]

#         save_path = "/media/queahren/DATA/DATA_A28_same/nums/" + imgname.split('.')[0] + '/'
#         save_path_data = "/media/queahren/DATA/DATA_A28_same/Y_Data/"
#         save_path_preview = "/media/queahren/DATA/DATA_A28_same/preview/"
        save_path = "/media/queahren/DATA/testData/测试数据/nums/" + imgname.split('.')[0] + '/'
        save_path_data = "/media/queahren/DATA/testData/测试数据/Y_Data/"
        save_path_preview = "/media/queahren/DATA/testData/测试数据/preview/"
#         save_path = "/media/queahren/DATA/MYDATA/nums/" + imgname.split('.')[0] + '/'
#         save_path_data = "/media/queahren/DATA/MYDATA/Y_Data/"
#         save_path_preview = "/media/queahren/DATA/MYDATA/preview/"
        
        if (os.path.exists(save_path) == False):
            os.mkdir(save_path)
        if (os.path.exists(save_path_data) == False):
            os.mkdir(save_path_data)    

        file0 = open(save_path_data + imgname.split('.')[0] + ".txt",'w+') 
        new_rects = []
        i = 0

        for rect in rects:

            x, y, w, h = rect
            roi = gray[y:y + h, x:x + w]
    #         print(w,h)     
            if (w > 4 and h < 20 and h > 5):
    #             cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 1, 1, 0);
                if w/h >= 1 and w < 22:
                    res = cv2.resize(roi, (2*w, 2*h))
                    res1 = resize(res[0:2*h, 0:w])
                    res2 = resize(res[0:2*h, w:2*w])

                    cv2.imwrite(save_path + '%d.jpg' % i, (res1)) 
                    num, prob = predImage(save_path + '%d.jpg' % i)
                    new_rects.append((x, y, round(w/2), h, num, i))
                    cv2.rectangle(img, (x, y), (x+round(w/2), y+h), (0, 0, 255), 1, 1, 0);
                    cv2.putText(img, '{:.0f}'.format(num), (x, y), cv2.FONT_HERSHEY_DUPLEX, h/25.0, (255, 0, 0))
                    i += 1

                    cv2.imwrite(save_path + '%d.jpg' % i, (res2))
                    num, prob = predImage(save_path + '%d.jpg' % i)
                    new_rects.append((x+round(w/2), y, round(w/2), h, num, i))
                    cv2.rectangle(img, (x+round(w/2), y), (x+w, y+h), (0, 0, 255), 1, 1, 0);
                    cv2.putText(img, '{:.0f}'.format(num), (x+round(w/2), y), cv2.FONT_HERSHEY_DUPLEX, h/25.0, (255, 0, 0))
                    i += 1

                elif w < 15:               
                    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 1, 1, 0);
                    res = resize(roi)
                    cv2.imwrite(save_path + '%d.jpg' % i, res)
                    num, prob = predImage(save_path + '%d.jpg' % i)
                    rect += tuple((num, i))
                    new_rects.append(rect)    
                    cv2.putText(img, '{:.0f}'.format(num), (x, y), cv2.FONT_HERSHEY_DUPLEX, h/25.0, (255, 0, 0))
                    i += 1

        cv2.imwrite(save_path_preview + imgname, img)    
        y_nums = getY(new_rects)
        p, y_num_max = calP(y_nums)
        y_nums, x_p = getX(new_rects)
        print(y_nums, file=file0)
        print(p, file=file0)
        print(y_num_max[1], file=file0)
        print(y_num_max[0], file=file0)
        print(x_p, file=file0)
        file0.close()
    



line_105.png
curve_0.png
curve_1.png
curve_10.png
curve_100.png
curve_101.png
curve_102.png
curve_103.png
curve_104.png
curve_105.png
curve_106.png
curve_107.png
curve_108.png
curve_109.png
curve_11.png
curve_110.png
curve_111.png
curve_112.png
curve_113.png
curve_90.png
curve_91.png
curve_92.png
curve_93.png
curve_94.png
curve_95.png
curve_96.png
curve_97.png
curve_98.png
curve_99.png
line_0.png
line_1.png
line_10.png
line_100.png
line_101.png
line_102.png
line_103.png
line_104.png
curve_250.png
curve_251.png
curve_252.png
curve_253.png
curve_254.png
curve_255.png
curve_256.png
curve_257.png
curve_258.png
curve_259.png
curve_26.png
curve_260.png
curve_261.png
curve_262.png
curve_263.png
curve_264.png
curve_265.png
curve_266.png
line_260.png
line_261.png
line_262.png
line_263.png
line_264.png
line_265.png
line_266.png
line_267.png
line_268.png
line_269.png
line_27.png
line_270.png
line_271.png
line_272.png
line_273.png
line_274.png
line_275.png
line_276.png
curve_420.png
curve_421.png


curve_320.png
curve_321.png
curve_322.png
curve_323.png
curve_324.png
curve_325.png
curve_326.png
curve_327.png
curve_328.png
curve_329.png
curve_33.png
curve_330.png
curve_331.png
curve_332.png
line_329.png
line_33.png
line_330.png
line_331.png
line_332.png
line_333.png
line_334.png
line_335.png
line_336.png
line_337.png
line_338.png
line_339.png
line_34.png
line_340.png
line_341.png
line_342.png
line_343.png
line_344.png
curve_334.png
curve_335.png
curve_336.png
curve_337.png
curve_338.png
curve_339.png
curve_34.png
curve_340.png
curve_341.png
curve_342.png
curve_343.png
curve_344.png
curve_345.png
curve_346.png
curve_347.png
curve_348.png
curve_349.png
line_122.png
line_14.png
line_157.png
line_174.png
line_191.png
line_208.png
line_225.png
line_242.png
line_26.png
line_277.png
line_294.png
line_310.png
line_328.png
line_345.png
line_362.png
line_38.png
line_397.png
line_413.png
line_44.png
line_457.png
line_474.png
line_491.png
line_62.png
line_80.png
line_346.png
line_347.png
line